<a href="https://colab.research.google.com/github/BaekTree/bookDataAnalysis/blob/master/craw.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install selenium
!apt-get update 
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin



     |████████████████████████████████| 911kB 12.1MB/s 
Get:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:6 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:7 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Ign:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:12 https://developer.download.nvidia.com/compu

In [24]:
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
import time
import requests
import pandas as pd
from pandas import DataFrame
import random

In [3]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
s =webdriver.Chrome('chromedriver',chrome_options=chrome_options)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: use options instead of chrome_options
  """


In [21]:
def fillSearch(start_mon, end_mon):
  s.get("http://www.g2b.go.kr/pt/menu/selectSubFrame.do?framesrc=/pt/menu/frameTgong.do?url=http://www.g2b.go.kr:8101/ep/tbid/tbidFwd.do?bidSearchType=1")
  frames=s.find_element_by_css_selector('#sub')
  f_name=frames.get_attribute('name')
  s.switch_to.frame(f_name)

  frames=s.find_element_by_css_selector('html > frameset > frame:nth-child(2)')

  s.switch_to.frame('main')

  typeBtn=s.find_element_by_xpath('//*[@id="bidSearchType2"]')
  typeBtn.click()

  workBtn=s.find_element_by_css_selector('#search > table > tbody > tr:nth-child(2) > td > div > label:nth-child(4)')
  workBtn.click()

  startDate=s.find_element_by_css_selector('#fromOpenBidDt')
  startDate.click()
  startDate.clear()
  startDate.send_keys(start_mon)

  endDate=s.find_element_by_css_selector('#toOpenBidDt')
  endDate.click()
  endDate.clear()
  endDate.send_keys(end_mon)

  s.find_element_by_xpath('//*[@id="area"]/option[@value="41"]').click()

  s.find_element_by_xpath('//*[@id="recordCountPerPage"]/option[@value="100"]').click()

  num=s.find_element_by_css_selector('#detailPrdnmNo')
  num.send_keys('5510151001')
  srchBtn=s.find_element_by_css_selector('#buttonwrap > div > a:nth-child(1)')
  srchBtn.click()
  

In [16]:
def crawl():
  

  tBody=s.find_elements_by_xpath('//*[@id="container"]/div[2]/table/tbody/tr')

  start_url='http://www.g2b.go.kr:8101/ep/result/prodBidResultDtl.do'
  # result_page = s.page_source
  ans_list = []
  tb_list = []
  for i in range(len(tBody)):
    print(i)
    css_slt="#container > div.results > table > tbody > tr:nth-child("+str(i+1)+") > td:nth-child(11) > div > a"
    # print(css_slt)
    dtlBtn=s.find_element_by_css_selector(css_slt)
    if dtlBtn.text == "개찰완료":
        r_num = random.randrange(1,6)
        time.sleep(3 + r_num)
        ans = dtlBtn.get_attribute('href')
        s_idx=ans.find('(')
        f_idx=ans.find(')')
        ans=ans[s_idx+1:f_idx].split(', ')
        temp = []
        for st in ans:
            removed = st.replace("\'", "")
            temp.append(removed)
        ans = temp
        # print(ans)
        ans_list.append(ans)

        # done="개찰완료".encode('euc-kr').decode('utf8')
        done="개찰완료".encode('euc-kr')
        payload={"bidno": ans[1], "bidseq": ans[2], "bidcate" : ans[3], "rebidno": ans[4], "progressInfo": done}
        # payload={'bidno': '20160514664', 'bidseq': '00', 'bidcate' : '0', 'rebidno': '0', 'progressInfo': done}
        # print(payload)
        # print({'bidno': '20160514664', 'bidseq': '00', 'bidcate' : '0', 'rebidno': '0', 'progressInfo': done})
        # print("-----------------------------\n\n" + json.dumps(payload) + "-----------------------------\n\n")
        response = requests.post(start_url, data=payload)
        # print(response.text)
        tb=pd.read_html(response.text)
        # print(type(tb))
        print(len(tb))
        df=tb[1]
        df.head()
        tb_list.append(df)
        # df.to_csv('./tb.csv',sep=',')
        # print(tb[1])
        # print(response.text)
        # break
        # s.back()
  print("done!")

In [22]:
st="2020/01/01"
end="2020/02/01"

In [19]:
def init():
  fillSearch(st, end)
  crawl()

In [15]:
fillSearch(st, end)

0
1
2
3
4
5
6
7
8
done!


In [25]:
## test
fillSearch(st, end)
tBody=s.find_elements_by_xpath('//*[@id="container"]/div[2]/table/tbody/tr')

start_url='http://www.g2b.go.kr:8101/ep/result/prodBidResultDtl.do'
# result_page = s.page_source
ans_list = []
tb_list = []
for i in range(len(tBody)):
  print(i)
  css_slt="#container > div.results > table > tbody > tr:nth-child("+str(i+1)+") > td:nth-child(11) > div > a"
  # print(css_slt)
  dtlBtn=s.find_element_by_css_selector(css_slt)
  if dtlBtn.text == "개찰완료":
      r_num = random.randrange(1,6)
      time.sleep(3 + r_num)
      ans = dtlBtn.get_attribute('href')
      s_idx=ans.find('(')
      f_idx=ans.find(')')
      ans=ans[s_idx+1:f_idx].split(', ')
      temp = []
      for st in ans:
          removed = st.replace("\'", "")
          temp.append(removed)
      ans = temp
      # print(ans)
      ans_list.append(ans)

      # done="개찰완료".encode('euc-kr').decode('utf8')
      done="개찰완료".encode('euc-kr')
      payload={"bidno": ans[1], "bidseq": ans[2], "bidcate" : ans[3], "rebidno": ans[4], "progressInfo": done}
      # payload={'bidno': '20160514664', 'bidseq': '00', 'bidcate' : '0', 'rebidno': '0', 'progressInfo': done}
      # print(payload)
      # print({'bidno': '20160514664', 'bidseq': '00', 'bidcate' : '0', 'rebidno': '0', 'progressInfo': done})
      # print("-----------------------------\n\n" + json.dumps(payload) + "-----------------------------\n\n")
      response = requests.post(start_url, data=payload)
      # print(response.text)
      tb=pd.read_html(response.text)
      # print(type(tb))
      print(tb)
      df=tb[1]
      df.head()
      tb_list.append(df)
      # df.to_csv('./tb.csv',sep=',')
      # print(tb[1])
      # print(response.text)
      break
      # s.back()

0
[        0  ...                                                  3
0  입찰공고번호  ...                                            Z026650
1     공고명  ...                       2020년 해밀도서관 도서(비도서 포함) 연간 구입
2    공고기관  ...                                              해밀도서관
3   공고담당자  ...                                                김세결
4  실제개찰일시  ...                                                보 기
5  적격심사결과  ...                                                NaN
6    유의사항  ...  ㅁ 사전판정 과정에서 부적격 처리된 업체의 투찰금액과 투찰률은 표시되지 않습니다. ...

[7 rows x 4 columns],    순위     사업자등록번호    업체명  대표자   투찰금액(원)  투찰률(%)   추첨번호                 투찰일시  비고
0   1  7262000134    책소리  박상기  29280000  73.477  13 10  2020/01/26 16:09:39 NaN
1   2  1304677114    희망북  안정훈  29600000  74.280  08 03  2020/01/20 20:54:42 NaN
2   3  1300594240   하나서적  정상구  32400000  81.306  01 09  2020/01/20 08:48:49 NaN
3   4  1302626316  신원종서점  이상석  33644000  84.428  07 14  2020/01/28 09:43:13 NaN
4   5  2133200058   제일문고  이영자  36148000  90.71

In [60]:
info = tb[0]
sup_name = info.iloc[2][1]
dmd_name = info.iloc[2][3]
print(sup_name)
print(dmd_name)
info
# info = info.T
# # info.iloc[0]
# info.columns = info.iloc[0]
# info = info.drop(info.index[0])
# # info.iloc[0]
# info

해밀도서관
해밀도서관


,0,1,2,3
0,입찰공고번호,20200111888-00 공고상세,참조번호,Z026650
1,공고명,2020년 해밀도서관 도서(비도서 포함) 연간 구입,2020년 해밀도서관 도서(비도서 포함) 연간 구입,2020년 해밀도서관 도서(비도서 포함) 연간 구입
2,공고기관,해밀도서관,수요기관,해밀도서관
3,공고담당자,김세결 (032-650-4652),집행관,김세결
4,실제개찰일시,2020/01/29 14:37,복수예비가 및예정가격,보 기
5,적격심사결과,NaN,NaN,NaN
6,유의사항,ㅁ 사전판정 과정에서 부적격 처리된 업체의 투찰금액과 투찰률은 표시되지 않습니다. ...,ㅁ 사전판정 과정에서 부적격 처리된 업체의 투찰금액과 투찰률은 표시되지 않습니다. ...,ㅁ 사전판정 과정에서 부적격 처리된 업체의 투찰금액과 투찰률은 표시되지 않습니다. ...
